In [9]:
random_state = 20
import pandas as pd
import json
def get_index_train_test(pipeline):
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn import preprocessing
    dataset, model_name ,classifier =tuple(pipeline.split('+'))
    df = pd.read_csv(f'Arquivos TCC/datasets/{dataset}.csv', sep=';')
    df = df.dropna()
    textos = df['text']
    labels = df['label']
    le = preprocessing.LabelEncoder()
    le.fit(labels)
    labels_enc = le.transform(labels)
    textos_train, textos_test, y_train, y_test = train_test_split(textos, labels_enc, test_size=0.10, random_state=random_state,stratify=labels_enc)
    return (list(textos_train.index),list(textos_test.index))


def get_df_test_pred(pipeline):
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn import preprocessing
    dataset, model_name ,classifier =tuple(pipeline.split('+'))
    df = pd.read_csv(f'Arquivos TCC/datasets/{dataset}.csv', sep=';')
    df = df.dropna()
    textos = df['text']
    labels = df['label']
    le = preprocessing.LabelEncoder()
    le.fit(labels)
    labels_enc = le.transform(labels)
    textos_train, textos_test, y_train, y_test = train_test_split(textos, labels_enc, test_size=0.10, random_state=random_state,stratify=labels_enc)
    df_test_pred = pd.read_csv(f'Arquivos TCC/test_set_predictions/{pipeline}.csv', sep=',')
    if dataset ==  'TC02_20news':
        df_test_pred['sentence'] = textos_test.to_list()
        df_test_pred['test_label'] = le.inverse_transform(df_test_pred['test_label'])
        df_test_pred['pred_label'] = le.inverse_transform(df_test_pred['pred_label'])
    if dataset == 'TC14_wos':
        with open("wos_label_map.json", "r") as read_file:
            label_map = json.load(read_file)
        df_test_pred['sentence'] = textos_test.to_list()  
        df_test_pred['test_label'] = df_test_pred['test_label'].apply(lambda x : label_map[str(x)])
        df_test_pred['pred_label'] = df_test_pred['pred_label'].apply(lambda x : label_map[str(x)])
    df_test_pred['index_dataset'] = get_index_train_test(pipeline)[1]

    #print(labels)
    return df_test_pred





In [55]:
best_pipes = ["TC02_20news+TFIDF+SVC","TC14_wos+TFIDF+RF"]
worst_pipes = ["TC02_20news+TFIDF+KNN","TC14_wos+TFIDF+KNN"]
df_test_pred = get_df_test_pred(best_pipes[0])
df_test_pred.head(10)

,test_label,pred_label,sentence,index_dataset
0,sci.crypt,sci.crypt,22 perhaps one way of getting away from this c...,11349
1,sci.space,sci.space,"all in all, im not certain that the single goa...",14439
2,rec.motorcycles,rec.motorcycles,there has been a running thread on the need to...,8531
3,rec.sport.hockey,rec.sport.hockey,door attendants at the joe louis arena have be...,10888
4,talk.politics.misc,talk.politics.misc,) ) ) the official and legal term for rape is ...,18935
5,sci.space,sci.space,this was known as journey to the far side of t...,14269
6,sci.electronics,sci.electronics,55 i am trying to build a circuit that detects...,12588
7,sci.med,sci.med,does anyone know how posionous centipedes and ...,13767
8,comp.os.ms-windows.misc,comp.sys.ibm.pc.hardware,"7 just as the title suggest, is it okay to do ...",2332
9,sci.med,sci.med,jb jb ron roth recommends: once you have your ...,13119


In [56]:
df_error = df_test_pred[df_test_pred['test_label']!=df_test_pred['pred_label']]
top_3_error_labels = df_error['test_label'].value_counts()[:3].index.tolist()
for label in top_3_error_labels:
    n = 5
    df_error_label = df_error[df_error['test_label']==label]
    top_n_misses = df_error_label['pred_label'].value_counts()[:n]
    print(f'True label: {label}')
    print('Most commom misunderstandings')
    print(top_n_misses)
    print('==========================')

True label: talk.religion.misc
Most commom misunderstandings
alt.atheism               23
talk.politics.misc        15
talk.politics.guns         4
soc.religion.christian     2
sci.med                    1
Name: pred_label, dtype: int64
True label: alt.atheism
Most commom misunderstandings
talk.religion.misc        27
sci.space                  2
talk.politics.mideast      1
soc.religion.christian     1
sci.electronics            1
Name: pred_label, dtype: int64
True label: talk.politics.misc
Most commom misunderstandings
talk.religion.misc       15
talk.politics.mideast     8
talk.politics.guns        5
rec.sport.baseball        1
Name: pred_label, dtype: int64


In [69]:
pd.options.display.max_colwidth = 300
df_dropado = df_test_pred.drop_duplicates(subset=['test_label'], keep='first')
df_dropado.sort_values(by=['test_label']).tail(10)


,test_label,pred_label,sentence,index_dataset
47,23,23,"Smokeless tobacco (ST) use among US high school males living in rural areas exceeds national levels. Subgroups at heightened risk of ST use have been identified, but less is known regarding ST decision-making within high-risk groups. The study objective was to describe rural adolescent males' pe...",4144
10,24,24,"Purpose of reviewIt has been proposed that biodiversity loss leads to reduced interaction between environmental and human microbiotas. This, in turn, may lead to immune dysfunction and impaired tolerance mechanisms in humans. That is, contact with environmental biodiversity is expected to protec...",5154
16,25,25,"A series of neurological and psychiatric symptoms occur after traumatic brain injury (TBI), with cognitive dysfunction being one of the most prominent sequela. Given that tau hyperphosphorylation is an important cause of cognitive impairment in patients of Alzheimer's disease, our present study ...",11661
28,26,9,"Cortical functioning within the default mode network (DMN) and salience network (SN) is altered in chronic pain patients. The mechanisms underlying these alterations are unknown, but a novel unexamined source is cross-network communication. Aberrant functional connectivity (FC) between the DMN a...",728
36,27,27,"With increasing rates of cancer patients undergoing radiation therapy, the treatment itself can cause patients significant amounts of anxiety and distress. This can be attributed to the diagnosis of the disease, lack of knowledge of what radiation therapy is, expectations and management of side ...",1032
6,28,28,This study aims to investigate the impacts of a digital learning module with guided peer feedback on students' domain-specific knowledge gain and their attitudinal change in the field of biotechnology and molecular life sciences. The extent to which the use of this module is appreciated by stude...,9068
11,29,29,"In the past decade mass spectrometry-based proteomics has greatly contributed to shaping our knowledge about Saccharomyces cerevisiae mitochondria, from the initial identification of novel essential components in purified protein complexes, to the actual characterization of the mitochondrial pro...",5214
30,30,30,"We review selected aspects of the interactions between protein and energy in human metabolism and nutrition. Following a short account of the underlying metabolic basis for the effects of energy on protein metabolism, the contribution made by whole body protein turnover to the metabolic rate is ...",1735
79,31,31,"We introduce a versatile carrier system for in vitro and in vivo immune stimulation based on soft matter DNA nanoparticles (NPs). The incorporation of lipid-modified nucleotides into DNA strands enables the formation of micelles of uniform size. In a single self-assembly step, the micelles can b...",9809
1,32,31,"Kawasaki disease is a multisystemic vasculitis that can result in coronary artery lesions. It predominantly affects young children and is characterized by prolonged fever, diffuse mucosal inflammation, indurative oedema of the hands and feet, a polymorphous skin rash and non-suppurative lymphade...",7526


In [70]:
data_catalog = pd.read_excel('/home/vinicius/Pré Processamento/Meta-data/Data.xlsx')

In [71]:
data_catalog.columns

Index(['Y1', 'Y2', 'Y', 'Domain', 'area', 'keywords', 'Abstract'], dtype='object')

In [75]:
label_map = {}
for row_index in range(len(df_dropado)):
    label_new = int(df_dropado['test_label'].values[row_index])
    try:
        label_wos = data_catalog[data_catalog['Abstract'].str.contains(df_dropado['sentence'].iloc[row_index][1:50].strip())]['area'].values[0]
        label_map[label_new] = label_wos.strip()
    except:
        print(f'Não achou o label_wos do {label_new}')
        print("Adicionado manualmente")

label_map[3] = 'Cryptography'
label_map[23] = 'Addiction'

/tmp/ipykernel_260/3465581413.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  label_wos = data_catalog[data_catalog['Abstract'].str.contains(df_dropado['sentence'].iloc[row_index][1:50].strip())]['area'].values[0]


Não achou o label_wos do 3
Adicionado manualmente
Não achou o label_wos do 23
Adicionado manualmente


In [76]:
label_map
with open('wos_label_map.json', 'w') as fp:
    json.dump(label_map, fp)

In [27]:
df_dropado['test_label'].values[row_index]

7

In [32]:
from sklearn import preprocessing
df = pd.read_csv(f'Arquivos TCC/datasets/TC02_20news.csv', sep=';')
df = df.dropna()
textos = df['text']
labels = df['label']
le = preprocessing.LabelEncoder()
le.fit(labels)
labels_enc = le.transform(labels)
label_map_20news = {}
for i,label in enumerate(labels.unique()):
    label_map_20news[i] = label

with open('20news_label_map.json', 'w') as fp:
    json.dump(label_map_20news, fp)

2

In [41]:
df_dropado['sentence'].iloc[row_index][0:100]

'This paper explores the large-signal and small-signal dynamics of a series-capacitor (SC) buck-type '

In [43]:
data_catalog[data_catalog['Abstract'].str.contains(df_dropado['sentence'].iloc[row_index][4:50])]['area']

40638     Digital control  
Name: area, dtype: object